In [1]:
#import the necessary modules
import pandas as pd
import numpy as np

In [2]:
# reading the train and test files for both the enquiry and account
acc_train = pd.read_csv(r'/Users/varsha/Downloads/test data/raw_account_70_new.csv',low_memory=False)
acc_test = pd.read_csv(r'/Users/varsha/Downloads/test data/raw_account_30_new.csv',low_memory=False)
enq_train = pd.read_csv(r'/Users/varsha/Downloads/test data/raw_enquiry_70_new.csv',low_memory=False)
enq_test = pd.read_csv(r'/Users/varsha/Downloads/test data/raw_enquiry_30_new.csv',low_memory=False)

In [3]:
# concat the train and test files
Ac_df = pd.concat([acc_train,acc_test]).reset_index(drop=True)
En_df = pd.concat([enq_train, enq_test]).reset_index(drop=True)

## Enquiry Feature Engineering

In [4]:
# converting the data in machine readable form for Enquiry
En_df['dt_opened']=pd.to_datetime(En_df['dt_opened']) # converted into datetime
En_df['enquiry_dt']=En_df['enquiry_dt'].fillna('01-Jan-2016') # fill missing date
En_df['enquiry_dt'] = pd.to_datetime(En_df['enquiry_dt']) # converted into datetime
En_df['upload_dt']=En_df['upload_dt'].fillna('01-Jan-2016') # fill missing date
En_df['upload_dt'] = pd.to_datetime(En_df['upload_dt']) #converted into datetime
En_df['days'] = En_df['upload_dt'].sub(En_df['enquiry_dt'], axis=0) # subtracted two date
En_df['days']= En_df['days'].astype('timedelta64[D]') #converted into datetime
En_df['days'][En_df['days'] == 0.0] = 10000
En_df['365_days'] = (En_df['days']<=365)
En_df['90_days'] = (En_df['days']<=90)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [5]:
#fill missing values
En_df['enq_purpose']=En_df['enq_purpose'].fillna(0) 
En_df['enq_amt']=En_df['enq_amt'].fillna(0)
En_df['openDays'] = En_df['dt_opened'].sub(En_df['enquiry_dt'], axis=0) #calculate the open days
En_df['openDays']= En_df['openDays'].astype('timedelta64[D]') #converted into datetime
En_df['openDays'][En_df['openDays'] < 0.0] = 0.0



/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
En_df_train = En_df[0:len(enq_train)] #spiltting the train set
En_df_test = En_df[len(enq_train):len(En_df)] #splitting the test set



In [7]:
aggregations = {
    '365_days':'sum',
    '90_days':'sum',
    'openDays': lambda x : sum(x)/len(x),
    'enq_amt':'sum'
}



In [8]:
En_df_train = En_df_train.groupby('customer_no', as_index=False).agg(aggregations) 
En_df_test = En_df_test.groupby('customer_no', as_index=False).agg(aggregations)
En_df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 5 columns):
customer_no    10240 non-null int64
365_days       10240 non-null float64
90_days        10240 non-null float64
openDays       10240 non-null float64
enq_amt        10240 non-null float64
dtypes: float64(4), int64(1)
memory usage: 480.0 KB


In [9]:
En_df_train.to_csv('/Users/prashant/Downloads/test data/trainEnquiry.csv', index=False) #save the train files as cvs
En_df_test.to_csv('/Users/prashant/Downloads/test data/testEnquiry.csv', index=False)   #save the train files as cvs

## Account File Feature Engineering

In [10]:
Ac_df.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,...,amt_past_due,paymenthistory1,paymenthistory2,paymt_str_dt,paymt_end_dt,creditlimit,cashlimit,rateofinterest,paymentfrequency,actualpaymentamount
0,10-Nov-15,12265,20-Oct-15,6,1,09-Jun-13,30-Jun-14,05-Jul-14,30-Sep-15,20900.0,...,NaN,"""""""STDSTDSTDXXXXXXXXXXXXXXXSTDXXXXXXXXXXXXXXXS...",NaN,01-Sep-15,01-Jul-14,NaN,NaN,NaN,NaN,NaN
1,10-Nov-15,12265,20-Oct-15,10,1,25-May-12,06-Sep-15,NaN,03-Oct-15,16201.0,...,NaN,"""""""0000000000000000000000000000000000000000000...","""""""000000000000000000000000000XXX0000000000000...",01-Oct-15,01-Nov-12,14000.0,1400.0,NaN,3.0,5603.0
2,10-Nov-15,12265,20-Oct-15,10,1,22-Mar-12,31-Aug-15,NaN,30-Sep-15,41028.0,...,NaN,"""""""0000000000000000000000000000000000000000000...","""""""0000000000000000000000000000000000000000000...",01-Sep-15,01-Oct-12,NaN,NaN,NaN,NaN,NaN
3,20-Jul-15,15606,09-Jul-15,10,1,13-Jan-06,NaN,26-Jul-07,31-Jan-09,93473.0,...,NaN,"""""""1200900600600600300000000000000000000000000...",NaN,01-Jul-07,01-Feb-06,NaN,NaN,NaN,NaN,NaN
4,20-Jul-15,15606,09-Jul-15,6,1,18-Jan-15,05-May-15,NaN,31-May-15,20250.0,...,NaN,"""""""000000000000000""""""",NaN,01-May-15,01-Jan-15,NaN,NaN,NaN,NaN,NaN


In [11]:
Ac_df['last_paymt_dt']=pd.to_datetime(Ac_df['last_paymt_dt']) #converted into datetime
Ac_df['opened_dt']=pd.to_datetime(Ac_df['opened_dt'])     #converted into datetime
Ac_df['openDays'] = Ac_df['last_paymt_dt'].sub(Ac_df['opened_dt'], axis=0) 
Ac_df['openDays']= Ac_df['openDays'].astype('timedelta64[D]')
# assigning the variable to new variable for further calculation
Ac_df['total_Days'] = Ac_df['openDays'] 
Ac_df['mean_Days'] = Ac_df['openDays']
Ac_df['total_creditlimit'] = Ac_df['creditlimit']
Ac_df['mean_creditlimit'] = Ac_df['creditlimit']
Ac_df['total_currentlimit'] = Ac_df['cur_balance_amt']
Ac_df['mean_currentlimit'] = Ac_df['cur_balance_amt']
# replacing missing value of creditlimit with high_credit_amt
Ac_df['creditlimit']= Ac_df['creditlimit'].fillna(Ac_df['high_credit_amt'])
# replacing missing value
fill_Col = ['paymentfrequency','cashlimit','amt_past_due','openDays',"total_creditlimit","rateofinterest","mean_creditlimit","total_Days","mean_Days"]



In [12]:
Ac_df.columns

Index(['dt_opened', 'customer_no', 'upload_dt', 'acct_type', 'owner_indic',
       'opened_dt', 'last_paymt_dt', 'closed_dt', 'reporting_dt',
       'high_credit_amt', 'cur_balance_amt', 'amt_past_due', 'paymenthistory1',
       'paymenthistory2', 'paymt_str_dt', 'paymt_end_dt', 'creditlimit',
       'cashlimit', 'rateofinterest', 'paymentfrequency',
       'actualpaymentamount', 'openDays', 'total_Days', 'mean_Days',
       'total_creditlimit', 'mean_creditlimit', 'total_currentlimit',
       'mean_currentlimit'],
      dtype='object')

In [13]:
#fill missing values
for i in fill_Col:
    Ac_df[i] = Ac_df[i].fillna(0)

In [14]:
# filling missing value of payment history
Ac_df["paymenthistory1"] = Ac_df['paymenthistory1'].fillna("XXX")
Ac_df["paymenthistory2"] = Ac_df['paymenthistory2'].fillna("XXX")
# payment boolean initialize,least_30dpd,max_30dpd
Ac_df['paymentBoolean'] = True
Ac_df['least_30dpd'] = 0.0
Ac_df['max_30dpd'] = 0.0
#cleaning the payment historys
n =3
acList = ['STD','XXX','SMA','SUB','DBT','LSS','"""','"""','ppp','iit']
acList2 = ['"""','"""','ppp','iit']
replaceList = ['STD','XXX','SMA','SUB','DBT','LSS']

In [15]:
#checking account in 0-29 dpd
string=""
Output=""
output=""
for row in range(0,len(Ac_df)):
    string=Ac_df.iloc[row]['paymenthistory1'] + Ac_df.iloc[row]['paymenthistory2']
    output = [string[q:q+n] for q in range(0, len(string), n)]
    Output = [item for item in output if item not in acList]
    if(len(Output)==0):
        Ac_df.ix[row,'paymentBoolean'] = False
    else:
        int_output = [int(i) for i in Output]
        booleanList = []
        for j in range(0,len(int_output)):
            if(int_output[j]>=30):
                booleanList.append(True)
            else:
                booleanList.append(False)
        if(True in booleanList):
            Ac_df.ix[row, 'paymentBoolean'] = False
        else:
            Ac_df.ix[row, 'paymentBoolean'] = True



/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  # Remove the CWD from sys.path while we load stuff.


In [16]:
# loop for calculating first and last 30+ dpd month
for row3 in range(0, len(Ac_df)):
    string3 = Ac_df.iloc[row3]['paymenthistory1'] + Ac_df.iloc[row3]['paymenthistory2']
    output3 = [string3[q3:q3 + n] for q3 in range(0, len(string3), n)]
    Ac_dfOutput3 = [item3 for item3 in output3 if item3 not in acList2]
    for z3, i3 in enumerate(Ac_dfOutput3):
        if i3 in replaceList:
            Ac_dfOutput3[z3] = '000'
    Ac_dfOutput3 = Ac_dfOutput3[::-1]
    if (len(Ac_dfOutput3) == 0):
        Ac_df.ix[row3, 'least_30dpd'] = 0.0
    else:
        int_output3 = [int(i4) for i4 in Ac_dfOutput3]
    indexList = []
    for index, value in enumerate(int_output3):
        if (value >= 30):
            indexList.append(index)

    if (len(indexList) == 0):
        Ac_df.ix[row3, 'least_30dpd'] = 0.0
    else:
        Ac_df.ix[row3, 'least_30dpd'] = indexList[0] + 1
        Ac_df.ix[row3,'max_30dpd'] = indexList[len(indexList)-1]+1

In [17]:
# payment history month length
Ac_df['paymenthistory1'] =Ac_df['paymenthistory1'].apply(len) -6
Ac_df['paymenthistory2'] = Ac_df['paymenthistory2'].apply(len) -6
Ac_df['monthlength'] = Ac_df['paymenthistory1']+ Ac_df['paymenthistory2']
# split the train and test
Ac_df_train = Ac_df[0:len(acc_train)]
Ac_df_test = Ac_df[len(acc_train):len(Ac_df)]
Ac_df_train.head()

,dt_opened,customer_no,upload_dt,acct_type,owner_indic,opened_dt,last_paymt_dt,closed_dt,reporting_dt,high_credit_amt,...,total_Days,mean_Days,total_creditlimit,mean_creditlimit,total_currentlimit,mean_currentlimit,paymentBoolean,least_30dpd,max_30dpd,monthlength
0,10-Nov-15,12265,20-Oct-15,6,1,2013-06-09,2014-06-30,05-Jul-14,30-Sep-15,20900.0,...,386.0,386.0,0.0,0.0,0,0,False,0.0,0.0,42
1,10-Nov-15,12265,20-Oct-15,10,1,2012-05-25,2015-09-06,NaN,03-Oct-15,16201.0,...,1199.0,1199.0,14000.0,14000.0,10390,10390,True,0.0,0.0,108
2,10-Nov-15,12265,20-Oct-15,10,1,2012-03-22,2015-08-31,NaN,30-Sep-15,41028.0,...,1257.0,1257.0,0.0,0.0,34420,34420,True,0.0,0.0,108
3,20-Jul-15,15606,09-Jul-15,10,1,2006-01-13,NaT,26-Jul-07,31-Jan-09,93473.0,...,0.0,0.0,0.0,0.0,0,0,False,14.0,19.0,51
4,20-Jul-15,15606,09-Jul-15,6,1,2015-01-18,2015-05-05,NaN,31-May-15,20250.0,...,107.0,107.0,0.0,0.0,13500,13500,True,0.0,0.0,12


In [18]:
#calculations
aggregations = {
    "cashlimit": lambda x : sum(x)/float(len(x)),
    "total_creditlimit":'sum',
    "total_currentlimit":'sum',
    "mean_creditlimit":lambda x : sum(x)/float(len(x)),
    "mean_currentlimit":lambda x : sum(x)/float(len(x)),
    'mean_Days':lambda x : sum(x)/len(x),
    'total_Days': lambda x : sum(x),
    'monthlength': lambda x: (sum(x)/3)/len(x),
    'paymentBoolean':lambda x : sum(x)/len(x),
    'least_30dpd': lambda x: max(x),
    'max_30dpd':lambda x : max(x)
}

In [19]:
# aggregations on train and test
Ac_df_train = Ac_df_train.groupby('customer_no', as_index=False).agg(aggregations)
Ac_df_test = Ac_df_test.groupby('customer_no', as_index=False).agg(aggregations)

In [20]:
Ac_df_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10240 entries, 0 to 10239
Data columns (total 12 columns):
customer_no           10240 non-null int64
cashlimit             10240 non-null float64
total_creditlimit     10240 non-null float64
total_currentlimit    10240 non-null int64
mean_creditlimit      10240 non-null float64
mean_currentlimit     10240 non-null float64
mean_Days             10240 non-null float64
total_Days            10240 non-null float64
monthlength           10240 non-null float64
paymentBoolean        10240 non-null float64
least_30dpd           10240 non-null float64
max_30dpd             10240 non-null float64
dtypes: float64(10), int64(2)
memory usage: 1.0 MB


In [21]:
# saving train and test
Ac_df_train.to_csv('/Users/varsha/Downloads/test data/trainAccount.csv', index=False)
Ac_df_test.to_csv('/Users/varsha/Downloads/test data/testAccount.csv', index=False)

